# Spark Core — базові приклади (самодостатній ноутбук)

Цей ноутбук демонструє базові операції з **PySpark**: створення `SparkSession`, роботу з DataFrame (CSV/JSON/Parquet/Text), RDD, простий `word count`, а також шаблони для Kafka та JDBC.

> Ноутбук зроблений так, щоб **працювати без зовнішніх файлів**: ми спочатку створюємо прикладові дані й записуємо їх у тимчасову папку, а потім зчитуємо назад.


## 1) Ініціалізація Spark

За замовчуванням використовується `local[*]`. Якщо у вас кластер, змініть `master` (наприклад, `spark://spark-master:7077`).


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
)

# Налаштування Spark
MASTER = "local[*]"  # або "spark://spark-master:7077"

spark = (
    SparkSession.builder
    .appName("spark_core_basics")
    .master(MASTER)
    .getOrCreate()
)

spark.version


In [ ]:
# Переконаємося, що Spark працює
spark.range(5).show()


## 2) Підготовка демо-даних

Створимо невеликий DataFrame і запишемо його у різні формати у `/tmp/spark_core_demo`.


In [ ]:
import os, shutil, json, pathlib

base_dir = pathlib.Path("/tmp/spark_core_demo")
if base_dir.exists():
    shutil.rmtree(base_dir)
base_dir.mkdir(parents=True, exist_ok=True)

people = spark.createDataFrame(
    [
        ("Alice", 30, 3500.5, ["python", "spark"]),
        ("Bob", 24, 2200.0, ["js", "react"]),
        ("Charlie", 41, 7800.0, ["sql", "spark", "etl"]),
        ("Daria", 35, 6100.3, ["ml", "python"]),
    ],
    schema=["name", "age", "salary", "skills"],
)

people.show(truncate=False)


In [ ]:
# Запис у CSV / JSON / Parquet
csv_path = str(base_dir / "people_csv")
json_path = str(base_dir / "people_json")
parquet_path = str(base_dir / "people_parquet")

(people
 .withColumn("skills", F.concat_ws(",", F.col("skills")))  # CSV з масивом незручно
 .write.mode("overwrite").option("header", True).csv(csv_path)
)

people.write.mode("overwrite").json(json_path)
people.write.mode("overwrite").parquet(parquet_path)

print("Written to:", csv_path, json_path, parquet_path)


In [ ]:
# Запис у Text (один рядок — один запис)
text_path = str(base_dir / "android_log_text")
text_df = spark.createDataFrame(
    [
        ("WindowManager: Something happened",),
        ("ActivityManager: Start proc 1234",),
        ("WindowManager: Another event",),
        ("SystemServer: Boot completed",),
    ],
    ["line"]
)

text_df.write.mode("overwrite").text(text_path)
print("Written to:", text_path)


## 3) Зчитування CSV у DataFrame

### 3.1) `inferSchema` + заголовки


In [ ]:
df_csv = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(csv_path)
)

df_csv.printSchema()
df_csv.show(truncate=False)


### 3.2) Власна схема (StructType)

Корисно, коли важлива точність типів або `inferSchema` занадто повільний.


In [ ]:
csv_schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("skills", StringType(), True),
])

df_csv_schema = (
    spark.read
    .option("header", True)
    .schema(csv_schema)
    .csv(csv_path)
)

df_csv_schema.printSchema()
df_csv_schema.show(truncate=False)


## 4) Зчитування JSON

### 4.1) Звичайний JSON


In [ ]:
df_json = spark.read.json(json_path)
df_json.printSchema()
df_json.show(truncate=False)


### 4.2) Приклад вкладеного JSON

Створимо файл зі вкладеним полем і покажемо `explode`.


In [ ]:
nested_path = str(base_dir / "nested_sales_json")

nested = spark.createDataFrame(
    [
        ("order-1", [{"sku": "A1", "qty": 2}, {"sku": "B7", "qty": 1}]),
        ("order-2", [{"sku": "A1", "qty": 1}]),
    ],
    schema=StructType([
        StructField("order_id", StringType(), True),
        StructField("items", ArrayType(StructType([
            StructField("sku", StringType(), True),
            StructField("qty", IntegerType(), True),
        ])), True),
    ])
)

nested.write.mode("overwrite").json(nested_path)

df_nested = spark.read.json(nested_path)
df_nested.printSchema()
df_nested.show(truncate=False)

df_items = (
    df_nested
    .select("order_id", F.explode("items").alias("item"))
    .select("order_id", F.col("item.sku").alias("sku"), F.col("item.qty").alias("qty"))
)

df_items.show(truncate=False)


## 5) Зчитування Parquet

Parquet зберігає схему та типи — зазвичай це найкращий формат для Data Lake.


In [ ]:
df_parquet = spark.read.parquet(parquet_path)
df_parquet.printSchema()
df_parquet.show(truncate=False)


## 6) Зчитування Text та базові RDD-операції

In [ ]:
df_text = spark.read.text(text_path).toDF("line")
df_text.show(truncate=False)

rdd = df_text.rdd.map(lambda row: row["line"])
rdd.take(5)


In [ ]:
# Приклади RDD
line_count = rdd.count()
first_line = rdd.first()
filtered = rdd.filter(lambda line: "WindowManager:" in line)

(line_count, first_line, filtered.collect())


## 7) Word Count (RDD)

Класика: розбити на слова → порахувати частоти → топ-N.


In [ ]:
import re

word_counts = (
    rdd
    .flatMap(lambda line: re.findall(r"[A-Za-z0-9_]+", line.lower()))
    .map(lambda w: (w, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)

word_counts.take(20)


## 8) Невеликий приклад з DataFrame API

Групування + агрегати.


In [ ]:
# Топ навичок (з DataFrame)
skills_df = (
    people
    .select("name", F.explode("skills").alias("skill"))
)

skills_df.groupBy("skill").agg(
    F.count("*").alias("cnt"),
    F.collect_list("name").alias("people")
).orderBy(F.desc("cnt")).show(truncate=False)


## 9) Шаблон: читання з Kafka (приклад)

> Потрібен пакет `org.apache.spark:spark-sql-kafka-0-10_2.12:<spark_version>`.
> У реальному оточенні додається через `--packages ...` або `spark.jars.packages`.

Цей блок — **шаблон**, він не запускається автоматично.


In [ ]:
# ПРИКЛАД (НЕ ЗАПУСКАТИ без налаштувань Kafka):
# kafka_df = (
#     spark.read.format("kafka")
#     .option("kafka.bootstrap.servers", "localhost:9092")
#     .option("subscribe", "my-topic")
#     .option("startingOffsets", "earliest")
#     .load()
# )
#
# messages = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp")
# messages.show(truncate=False)


## 10) Шаблон: JDBC з partitioning (приклад)

Це пришвидшує читання великих таблиць.

Цей блок — **шаблон**, він не запускається автоматично.


In [ ]:
# ПРИКЛАД (НЕ ЗАПУСКАТИ без драйвера та доступу до БД):
# jdbc_url = "jdbc:postgresql://localhost:5432/mydb"
# props = {"user": "user", "password": "pass", "driver": "org.postgresql.Driver"}
#
# df_jdbc = (
#     spark.read.jdbc(
#         url=jdbc_url,
#         table="public.big_table",
#         column="id",      # колонка для розбиття
#         lowerBound=1,
#         upperBound=10_000_000,
#         numPartitions=16,
#         properties=props
#     )
# )
# df_jdbc.select("id").count()


## 11) Завершення

In [ ]:
spark.stop()
print("Spark session stopped.")
